## Training loop (WIP)

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from tfrecord.torch.dataset import MultiTFRecordDataset

In [10]:
import torch 
import torchgeo
import json
import numpy as np
# import pandas as pd
from models.resnet18 import ResNet18
# from data_handlers.datasets.csv_dataset import CustomDatasetFromCSV 
from tqdm import tqdm

1.(WIP) Custom Dataloader 

In [21]:
import glob

train_files = glob.glob('data/datasets/*/*2016*.tfrecord')
val_files = glob.glob('data/datasets/*/*2011*.tfrecord')
                        
print(train_files)

['data/datasets/Landsat/madagascar_2016_00.tfrecord']


In [23]:
import tensorflow as tf

2023-03-20 15:50:41.272671: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-20 15:50:41.272828: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-20 15:51:03.094220: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-20 15:51:03.101266: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-20 15:51:03.101357: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

In [25]:
train_feature_description = {    
    'LON': tf.io.FixedLenFeature([], tf.float32),
    'TEMP1': tf.io.FixedLenFeature([], tf.float32),
    'NIR':  tf.io.FixedLenFeature([], tf.float32),
    'year': tf.io.FixedLenFeature([], tf.float32), 
    'system:index': tf.io.FixedLenFeature([], tf.float32),
    'lon': tf.io.FixedLenFeature([], tf.float32),
    'GREEN': tf.io.FixedLenFeature([], tf.float32), 
    'RED': tf.io.FixedLenFeature([], tf.float32),
    'LAT': tf.io.FixedLenFeature([], tf.float32),
    'SWIR2': tf.io.FixedLenFeature([], tf.float32),
    'wealthpooled': tf.io.FixedLenFeature([], tf.float32),
    'lat': tf.io.FixedLenFeature([], tf.float32),
    'NIGHTLIGHTS': tf.io.FixedLenFeature([], tf.float32),
    'BLUE': tf.io.FixedLenFeature([], tf.float32),
    'country': tf.io.FixedLenFeature([], tf.float32), 
    'SWIR1': tf.io.FixedLenFeature([], tf.float32),
}

In [26]:
def _parse_image_function(example_proto):
    return tf.io.parse_single_example(example_proto, train_feature_description)

In [ ]:
# train_ids = []
# train_class = []
# train_images = []for i in train_files:
#     train_image_dataset = tf.data.TFRecordDataset(i)train_image_dataset = train_image_dataset.map(_parse_image_function)ids = [str(id_features['id'].numpy())[2:-1] for id_features in train_image_dataset] # [2:-1] is done to remove b' from 1st and 'from last in train id names
#     train_ids = train_ids + idsclasses = [int(class_features['class'].numpy()) for class_features in train_image_dataset]
#     train_class = train_class + classesimages = [image_features['image'].numpy() for image_features in train_image_dataset]
#     train_images = train_images + images

3. Pre-made Dataset

In [27]:
PATH = "./data/datasets/landsat/{}.tfrecord"
splits = {
    "madagascar_1997_00": 0.25,
    "madagascar_2011_00": 0.25,
    "madagascar_2013_00": 0.25,
    "madagascar_2016_00": 0.25,
}
description = {
    'LON': 'float',
    'TEMP1':'float',
    'NIR': 'float',
    'year':'float', 
    'system:index':'byte',
    'lon':'float',
    'GREEN':'float', 
    'RED':'float',
    'LAT':'float',
    'SWIR2':'float',
    'wealthpooled':'float',
    'lat':'float',
    'NIGHTLIGHTS':'float',
    'BLUE':'float',
    'country':'byte', 
    'SWIR1':'float',
}
dataset = MultiTFRecordDataset(
    PATH, 
    None, 
    splits, 
    description
)
loader  = torch.utils.data.DataLoader(dataset,batch_size=16)

2. Model

In [ ]:
with open('configs/resnet18_ms.json') as f:
    config = json.load(f)
    model = ResNet18(config)
    model.load_checkpoints('models/checkpoints/resnet18_wang_ckpt.pth')

In [ ]:
model.train()

In [8]:
data = next(iter(loader))
Y = data['wealthpooled']
print(Y, data.shape())

AttributeError: 'dict' object has no attribute 'shape'